# 7.4 Задание на построение линейной регрессии

Постройте линейную регрессионную модель зависимости веса новорожденного от расы и образования матери, срока беременности, а также информации о курении матери во время беременности.

In [100]:
import scipy.stats as sps
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

In [101]:
data = pd.read_csv(r'https://stepik.org/media/attachments/lesson/28461/weights.txt', sep='\t')
data

,race,educ,smoke,preg,weight
0,1,8,0,39,3670
1,1,9,1,38,2775
2,1,9,0,38,2680
3,1,9,0,37,2699
4,1,9,1,38,2557
...,...,...,...,...,...
95,0,16,0,38,3097
96,1,16,0,39,4091
97,1,16,0,40,3381
98,0,16,0,40,2727


In [102]:
weight = np.array(data['weight']).reshape(-1, 1)
race = np.array(data['race'])
educ = np.array(data['educ'])
smoke = np.array(data['smoke'])
preg = np.array(data['preg'])
n = len(data)
k = 4

In [103]:
linreg = LinearRegression()
X = pd.DataFrame({'race': race, 'educ': educ, 'smoke': smoke, 'preg': preg})
linreg.fit(y=weight, X=X)

LinearRegression()

betas

In [104]:
beta0 = linreg.intercept_
beta1 = linreg.coef_[0][0]
beta2 = linreg.coef_[0][1]
beta3 = linreg.coef_[0][2]
beta4 = linreg.coef_[0][3]
beta0, beta1

(array([-2841.37767615]), -211.57152450692422)

Найдите оценку дисперсии ошибок наблюдений

In [105]:
rss = sum(map(lambda x: (x[0] - (beta0 + beta1*x[1] + beta2*x[2] + beta3*x[3] + beta4*x[4]))**2, zip(weight,  X['race'], X['educ'], X['smoke'], X['preg'])))
S2 = rss[0]/(n - k - 1)
S = np.sqrt(S2)
S2

193390.2861174859

Найдите точность 95% доверительного интервала для параметра при сроке беременности

In [106]:
A = np.matrix(pd.DataFrame({'1': np.ones(n),'race': race, 'educ': educ, 'smoke': smoke, 'preg': preg}))
AT = A.transpose()
alpha = 0.05
sps.t.ppf(1-alpha/2, n-k-1)*S*np.sqrt(np.linalg.pinv(np.dot(AT, A))[4,4])

55.38424956674616

Найдите коэффициент детерминации

In [107]:
x = np.average(weight)
R2 = (1 - rss/sum(map(lambda pr: (pr - x)**2, weight))[0])[0]
R2

0.3364652117987227

Найдите p-значение и примите статистическое решение о значимости модели 

In [108]:
import statsmodels.api as sm

Y = data.weight
X = data[['race', 'educ', 'smoke', 'preg']]
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
a = 0.05
f_pvalue = results.f_pvalue
print(round(f_pvalue, 2), 1 if 1 - f_pvalue > a else 0)

0.0 1


Чему равен коэффициент Акаике построенной модели?

In [109]:
AIC = 2*k + n*(np.log(rss/n) + 1)
AIC

array([1320.11723395])

Рассмотрите модель зависимости веса новорожденного от расы и срока беременности. Чему равен коэффициент Акаике в данной модели?

In [112]:
k_new = 2
X_new = pd.DataFrame({'race': race, 'preg': preg})
linreg_new = LinearRegression()
linreg_new.fit(y=weight, X=X_new)
beta0_n = linreg_new.intercept_
beta1_n = linreg_new.coef_[0][0]
beta2_n = linreg_new.coef_[0][1]
rss_n = sum(map(lambda x: (x[0] - (beta0_n + beta1_n*x[1] + beta2_n*x[2]))**2, zip(weight,  X_new['race'], X_new['preg'])))
# S2 = rss[0]/(n - k - 1)
# S = np.sqrt(S2)
AIC_n = 2*k_new + n*(np.log(rss_n/n) + 1)
AIC_n

array([1316.6439727])